<a href="https://colab.research.google.com/github/yongjun1994/branchTest/blob/main/6_%EC%84%9C%EC%9A%B8%EC%8B%9C_%EA%B3%B5%EA%B3%B5%EC%9E%90%EC%A0%84%EA%B1%B0_%EC%8B%A4%EC%8B%9C%EA%B0%84_%EB%8C%80%EC%97%AC%EC%A0%95%EB%B3%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 서울 열린데이터 광장**
서울 열린데이터 광장(Seoul Open Data Plaza)은 서울시에서 운영하는 공공데이터 개방 플랫폼입니다. 시민, 연구자, 기업 등이 서울시에서 생성한 다양한 공공데이터를 자유롭게 활용할 수 있도록 제공하고 있습니다. 이를 통해 데이터 기반의 창의적인 아이디어와 혁신을 촉진하며, 시민들의 정보 접근성을 높이고 공공서비스를 개선하는 데 기여하고 있습니다.
* [공식 사이트](https://data.seoul.go.kr/)

In [ ]:
import requests # 백엔드 만들어 놓고 접속하기 위해 씀
import folium
import json # json 형식으로 데이터 가져오기
import pandas as pd # 데이터 프레임 다루기 위해
import warnings # warning 감추기 위해
warnings.filterwarnings('ignore')

In [ ]:
base_url = 'http://openapi.seoul.go.kr:8088/7a4e4d4353666d643330564d68575a/json/bikeList/1/1000/'
response = requests.get(base_url)
# print(response) # <Response [200]> : 정상적인 접근
json_data = response.json(3)
json_data

{'rentBikeStatus': {'list_total_count': 1000,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '15',
    'stationName': '102. 망원역 1번출구 앞',
    'parkingBikeTotCnt': '14',
    'shared': '93',
    'stationLatitude': '37.55564880',
    'stationLongitude': '126.91062927',
    'stationId': 'ST-4'},
   {'rackTotCnt': '14',
    'stationName': '103. 망원역 2번출구 앞',
    'parkingBikeTotCnt': '4',
    'shared': '29',
    'stationLatitude': '37.55495071',
    'stationLongitude': '126.91083527',
    'stationId': 'ST-5'},
   {'rackTotCnt': '13',
    'stationName': '104. 합정역 1번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55073929',
    'stationLongitude': '126.91508484',
    'stationId': 'ST-6'},
   {'rackTotCnt': '5',
    'stationName': '105. 합정역 5번출구 앞',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.55000687',
    'stationLongitude': '126.91482544',
    'stationId': 'ST-7'},
   {'rackTotCnt': '12',
 

In [ ]:
json_data.get('rentBikeStatus', {}).get("RESULT", {}).get("CODE", "")

'INFO-000'

In [ ]:
json_data['rentBikeStatus']['RESULT']['CODE']

'INFO-000'

In [ ]:
import time

def fetch_bike_data():
    base_url = 'http://openapi.seoul.go.kr:8088/7a4e4d4353666d643330564d68575a/json/bikeList/'
    start = 1
    end = 1000
    step = 1000
    data_frames = []

    while True:
        url = f'{base_url}{start}/{end}/'
        response = requests.get(url)
        time.sleep(1)
        if response.status_code != 200:
            print(f'Status Code: {response.status_code}')
            break
        json_data = response.json()
        try:
            rent_bike_status = json_data['rentBikeStatus']
            result_code = rent_bike_status['RESULT']['CODE']
        except KeyError:
            print('JSON 오류')
            break

        if result_code == 'INFO-200':
            print('데이터 없음')
            break
        elif result_code == 'INFO-000':
            print(f'시작: {start} 끝: {end}')
            try:
                bike_data = rent_bike_status['row']
                if bike_data:
                    df = pd.DataFrame(bike_data)
                    data_frames.append(df)
            except KeyError:
                print('데이터를 찾을 수 없음')
        else:
            print(f'result code: {result_code}')
            break

        start += step
        end += step

    if data_frames:
        final_df = pd.concat(data_frames, ignore_index=True)
        return final_df

    else:
        return pd.DataFrame() # 빈 데이터 프레임을 리턴

In [ ]:
bike_data_df = fetch_bike_data() # 함수를 실행, 데이터합쳤을때 2717개 들어옴
bike_data_df

시작: 1 끝: 1000
시작: 1001 끝: 2000
시작: 2001 끝: 3000
JSON 오류


,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,15,100,37.55564880,126.91062927,ST-4
1,14,103. 망원역 2번출구 앞,3,21,37.55495071,126.91083527,ST-5
2,13,104. 합정역 1번출구 앞,0,0,37.55073929,126.91508484,ST-6
3,5,105. 합정역 5번출구 앞,0,0,37.55000687,126.91482544,ST-7
4,12,106. 합정역 7번출구 앞,4,33,37.54864502,126.91282654,ST-8
...,...,...,...,...,...,...,...
2712,10,6172. 가양5단지아파트,2,20,37.56447983,126.85464478,ST-3281
2713,10,6173. 서울자동차운전전문학원,2,20,37.54613495,126.82106018,ST-3284
2714,7,6176. 유광사 여성병원 앞,1,14,37.54344940,126.83891296,ST-3327
2715,10,6177. 마곡롯데캐슬르웨스트,5,50,37.56356049,126.82462311,ST-3324


In [ ]:
bike_data_df.info() # object(7) : 전부 String

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   rackTotCnt         2717 non-null   object
 1   stationName        2717 non-null   object
 2   parkingBikeTotCnt  2717 non-null   object
 3   shared             2717 non-null   object
 4   stationLatitude    2717 non-null   object
 5   stationLongitude   2717 non-null   object
 6   stationId          2717 non-null   object
dtypes: object(7)
memory usage: 148.7+ KB


In [ ]:
'''
rackTotCnt	거치대개수
parkingBikeTotCnt	자전거주차총건수
shared	거치율
stationLatitude	위도
stationLongitude	경도
stationId	대여소ID
stationName	대여소이름
'''
bike_data_df.columns

Index(['rackTotCnt', 'stationName', 'parkingBikeTotCnt', 'shared',
       'stationLatitude', 'stationLongitude', 'stationId'],
      dtype='object')

In [ ]:
bike_data_df['stationLongitude'] = bike_data_df['stationLongitude'].astype(float)
bike_data_df['stationLatitude'] = bike_data_df['stationLatitude'].astype(float)

In [ ]:
bike_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rackTotCnt         2717 non-null   object 
 1   stationName        2717 non-null   object 
 2   parkingBikeTotCnt  2717 non-null   object 
 3   shared             2717 non-null   object 
 4   stationLatitude    2717 non-null   float64
 5   stationLongitude   2717 non-null   float64
 6   stationId          2717 non-null   object 
dtypes: float64(2), object(5)
memory usage: 148.7+ KB


In [ ]:
bike_map = folium.Map(location=[bike_data_df['stationLatitude'].mean(),
                                bike_data_df['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_data_df.iterrows():
    popup_str = '{} 자전거주차총건수:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map